In [1]:
# Dataframe from NSE Option Chains

# STATUS: Complete

import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np

symbol = 'ACC'  # Sample symbol

url_base = "https://nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol="
url_end = '&date='
url = url_base + symbol

page = requests.get(url)
doc = lh.fromstring(page.content)

#...    Get the Expiries ...#
#____________________________

the_path= "//form"   # Get the form content. There are two forms
expiries = doc.xpath(the_path)[0].text_content()
strip_chars = [ord('\n'), ord('\xa0'), ord('\t'), ord('\r')] # characters to be stripped
char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

sym_exp = expiries.translate(char_table).split() # split converts the translated dict into list

# Get the expiries only, from the table
expiry = [sym_exp[k+i] 
 for k, v in enumerate(sym_exp) 
 if v in 'Select' 
 for i in range(len(sym_exp) - k)][1:]

# Convert expiry to datetime
expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

#...   Get the Options Data .... #
#_________________________________

tbl_path = "//*[@id='octable']"  # xpath for the table

# The headers
opt_head = ['cOI', 'cChnginOI', 'cVolume', 'cIV', 'cLTP', 
            'cNetChng', 'cBidQty', 'cBidPrice', 'cAskPrice', 'cAskQty', 'Strike', 
            'pBidQty', 'pBidPrice', 'pAskPrice', 'pAskQty', 'pNetChng', 'pLTP', 
            'pIV', 'pVolume', 'pChnginOI', 'pOI']

opt_data = [tr.text_content() for table in doc.xpath(tbl_path) for tr in table[1:]]

df1 = pd.DataFrame([d.split() for d in opt_data], columns=opt_head)

df2 = df1.replace(',', '', regex=True) # Remove comma from numbers
df2 = df2.apply(pd.to_numeric, errors='coerce') # Convert to numeric
df2.insert(0, 'Expiry', expiry_dt[1]) # Insert the Expiry column
df2.insert(0, 'Symbol', symbol) # Insert the Symbol

# Rearrange the columns
cols_beginning = ['Symbol', 'Expiry', 'Strike']
df2 = df2[cols_beginning + [c for c in df2 if c not in cols_beginning]]

In [2]:
df2

,Symbol,Expiry,Strike,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,cBidQty,...,pBidQty,pBidPrice,pAskPrice,pAskQty,pNetChng,pLTP,pIV,pVolume,pChnginOI,pOI
0,ACC,2018-12-27,1140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.00,400.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ACC,2018-12-27,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1200.0,0.05,NaN,NaN,NaN,0.05,NaN,NaN,NaN,2400.0
2,ACC,2018-12-27,1220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1200.0,0.05,NaN,NaN,NaN,1.25,NaN,NaN,NaN,800.0
3,ACC,2018-12-27,1240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,800.0,0.10,NaN,NaN,NaN,1.75,NaN,NaN,NaN,800.0
4,ACC,2018-12-27,1260.0,400.0,NaN,NaN,NaN,209.60,NaN,NaN,...,800.0,0.80,2.00,800.0,NaN,1.60,NaN,NaN,NaN,10800.0
5,ACC,2018-12-27,1280.0,2800.0,NaN,NaN,NaN,99.75,NaN,1200.0,...,400.0,0.60,NaN,NaN,NaN,2.60,NaN,NaN,NaN,3600.0
6,ACC,2018-12-27,1300.0,6800.0,NaN,2.0,NaN,192.35,25.05,3600.0,...,1200.0,1.00,2.00,400.0,0.65,1.65,47.32,4.0,-1200.0,26000.0
7,ACC,2018-12-27,1320.0,400.0,NaN,NaN,NaN,84.00,NaN,1600.0,...,800.0,0.40,NaN,NaN,NaN,3.90,NaN,NaN,NaN,5200.0
8,ACC,2018-12-27,1340.0,2000.0,NaN,4.0,38.58,155.20,74.45,2400.0,...,1600.0,0.35,4.90,800.0,0.70,2.00,40.17,4.0,-1200.0,6800.0
9,ACC,2018-12-27,1360.0,3600.0,-2400.0,10.0,36.90,135.95,-14.90,3600.0,...,1200.0,1.05,1.60,400.0,-2.20,1.60,32.27,11.0,-2000.0,7200.0
